# Pack AnsweringModels

Pack NN, tokenaizers, index, passages

In [1]:
# plotly standard imports
import plotly.graph_objs as go
import chart_studio.plotly as py

# Cufflinks wrapper on plotly
import cufflinks

# Data science imports
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 30

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from plotly.offline import iplot, init_notebook_mode
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

# Set global theme
cufflinks.set_config_file(world_readable=True, theme='pearl')

In [2]:
ARCHIVE_FOLDER = './answer_data'
TOKENIZER_FILENAME = f'{ARCHIVE_FOLDER}/tokenizer'
MODEL_FILENAME = f'{ARCHIVE_FOLDER}/model'

## Save model and tokenaizer

In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

qa_s2s_tokenizer = AutoTokenizer.from_pretrained('yjernite/bart_eli5', cache_dir='./tokenaizers')
qa_s2s_model = AutoModelForSeq2SeqLM.from_pretrained('yjernite/bart_eli5', cache_dir='./models')
_ = qa_s2s_model.eval()

In [4]:
qa_s2s_tokenizer.save_pretrained(TOKENIZER_FILENAME)
qa_s2s_model.save_pretrained(MODEL_FILENAME)

('./answer_data/tokenizer/tokenizer_config.json',
 './answer_data/tokenizer/special_tokens_map.json',
 './answer_data/tokenizer/vocab.json',
 './answer_data/tokenizer/merges.txt',
 './answer_data/tokenizer/added_tokens.json',
 './answer_data/tokenizer/tokenizer.json')

### Check model and tokenaizer

In [6]:
qa_s2s_tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_FILENAME, use_fast=False)
qa_s2s_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_FILENAME)
_ = qa_s2s_model.eval()

## Pack and upload

In [18]:
import os
import tarfile

def pack_folder(source_dir = '', file_name = ''):
    
    with tarfile.open(file_name + '.tar.gz', 'w:gz') as f:
        f.add(source_dir, arcname = os.path.basename(source_dir))
    
    return f"{os.getcwd()}/{file_name}.tar.gz"

model_filename = pack_folder(ARCHIVE_FOLDER, 'search_data')
print('model packed to', model_filename)

model packed to /work/search_data.tar.gz


In [20]:
from dotenv import load_dotenv
load_dotenv()

True

In [22]:
import boto3


def upload_file(model_path='', s3_bucket='', s3_filename=''):
    s3 = boto3.session.Session()
    client = s3.client('s3')
    return client.upload_file(model_path, s3_bucket, s3_filename)
    
upload_file(model_filename, 'asqa-answer-models', 'BART/en.tar.gz')